<a href="https://colab.research.google.com/github/cswcjt/Dacon-Oil/blob/main/second_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from google.colab import drive
drive.mount('/content/drive')
base_path = "/content/drive/MyDrive/fastcamp/datas/open/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install optuna

In [60]:
%cd "/content/drive/MyDrive/fastcamp/datas/open/"
from dacon_ensemble import BinaryCalssifier, Regressor
from dacon_preprocssing import Preprocessing

from sklearn.decomposition import PCA # 차원축소
from sklearn.model_selection import train_test_split 
from imblearn.under_sampling import * # 임벨런스
from imblearn.over_sampling import * # 임벨런스
from imblearn.combine import * # 임벨런스

import time
from tqdm import tqdm
import copy
import pandas as pd

/content/drive/MyDrive/fastcamp/datas/open


목적 : 각 그룹으로 나누고 날짜별로 비정상이 뜨는 경우를 조사해보자

In [121]:
train = pd.read_csv(base_path + "train.csv", encoding='cp949').dropna(axis=1)
test = pd.read_csv(base_path + "test.csv", encoding='cp949')
submission = pd.read_csv(base_path + "sample_submission.csv", encoding='cp949')

In [122]:
train.COMPONENT_ARBITRARY.unique()

array(['COMPONENT3', 'COMPONENT2', 'COMPONENT1', 'COMPONENT4'],
      dtype=object)

In [123]:
df_list = []
for criteria in sorted(train.COMPONENT_ARBITRARY.unique()):
    print(criteria)
    df_list.append(train[train.COMPONENT_ARBITRARY == criteria])

COMPONENT1
COMPONENT2
COMPONENT3
COMPONENT4


In [124]:
COMPONENT1_df = df_list[0]
COMPONENT2_df = df_list[1]
COMPONENT3_df = df_list[2]
COMPONENT4_df = df_list[3]

In [125]:
train = train.sort_values(by="SAMPLE_TRANSFER_DAY")
train.loc[(train.SAMPLE_TRANSFER_DAY == 25) & (train.Y_LABEL == 0), ]
#train.loc[(train.SAMPLE_TRANSFER_DAY == 25) & (train.Y_LABEL == 1), ]

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,PQINDEX,S,SB,SI,SN,TI,V,V40,ZN,Y_LABEL
12941,TRAIN_12941,COMPONENT2,2208,2017,25,1408,0,2,0,0,...,5,5603,0,14,0,0,0,68.0,158,0
3560,TRAIN_03560,COMPONENT1,1191,2016,25,391,0,0,0,0,...,24,8288,2,4,2,0,0,148.4,822,0
6181,TRAIN_06181,COMPONENT1,3457,2015,25,200,0,9,10,0,...,14,10200,0,18,0,0,0,107.7,1043,0
2347,TRAIN_02347,COMPONENT3,2208,2017,25,200,0,0,112,1,...,3475,3240,0,14,8,0,0,58.2,1149,0
13369,TRAIN_13369,COMPONENT2,3146,2016,25,200,0,0,2,0,...,11,1225,0,2,0,0,0,44.1,621,0
9520,TRAIN_09520,COMPONENT3,2239,2008,25,375,0,6,130,1,...,687,16440,3,13,2,0,0,141.5,148,0
12506,TRAIN_12506,COMPONENT1,2717,2013,25,200,0,4,126,0,...,6,8112,0,4,0,0,0,85.1,1211,0
13521,TRAIN_13521,COMPONENT1,3146,2016,25,200,0,0,1,0,...,8,4270,0,4,0,0,0,94.9,1166,0
6447,TRAIN_06447,COMPONENT3,2208,2017,25,708,0,1,109,1,...,147,6530,0,47,2,0,0,116.3,947,0
238,TRAIN_00238,COMPONENT3,3078,2016,25,200,0,4,5,3,...,594,16910,0,6,0,0,0,167.4,28,0


In [126]:
#day_list

In [ ]:
day_list = train.sort_values(by="SAMPLE_TRANSFER_DAY").SAMPLE_TRANSFER_DAY.unique()

all_clear_list = []
for day in day_list:
    numerator = len(train.loc[(train.SAMPLE_TRANSFER_DAY == day) & (train.Y_LABEL == 1), ])
    denominator = len(train.loc[(train.SAMPLE_TRANSFER_DAY == day),])
    print(f"ratio of {day} = ", (numerator/denominator))
    if (numerator/denominator) == 0:
        all_clear_list.append(day)

for normal in all_clear_list: 
    train2 = train.drop(train[train.SAMPLE_TRANSFER_DAY == normal].index)

In [131]:
for normal in all_clear_list: 
    train2 = train.drop(train[train.SAMPLE_TRANSFER_DAY == normal].index)

In [ ]:
train

In [ ]:
train2

In [ ]:
concat_target = []
for day in all_clear_list:
    #print(day)
    concat_target.append(COMPONENT1_df[COMPONENT1_df.SAMPLE_TRANSFER_DAY == day])

df = pd.concat(concat_target, ignore_index=True)

In [107]:
df

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,PQINDEX,S,SB,SI,SN,TI,V,V40,ZN,Y_LABEL
0,TRAIN_01704,COMPONENT1,1518,2016,25,200,0,5,4,4,...,25,6342,2,11,2,0,0,92.4,992,0
1,TRAIN_01939,COMPONENT1,1182,2011,25,382,0,17,3,3,...,7,4679,0,10,5,0,0,123.0,1483,0
2,TRAIN_03560,COMPONENT1,1191,2016,25,391,0,0,0,0,...,24,8288,2,4,2,0,0,148.4,822,0
3,TRAIN_05277,COMPONENT1,5051,2018,25,375,0,2,544,0,...,12,3398,0,11,0,0,0,89.5,1260,0
4,TRAIN_05625,COMPONENT1,1191,2016,25,391,0,2,2,2,...,11,7051,2,6,2,0,0,100.3,1283,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,TRAIN_12344,COMPONENT1,1666,2009,134,550,0,1,81,0,...,51,7034,3,9,5,0,0,99.1,1183,0
59,TRAIN_10223,COMPONENT1,1721,2011,163,200,0,3,10,2,...,16,12120,0,6,0,0,0,100.0,1307,0
60,TRAIN_01754,COMPONENT1,1875,2011,175,200,0,14,146,0,...,15,4251,0,14,12,0,0,86.2,1099,0
61,TRAIN_09331,COMPONENT1,1000,2009,203,440,0,7,2,0,...,11,8011,1,11,2,0,0,102.6,1225,0


In [ ]:
test

In [111]:
test[test.COMPONENT_ARBITRARY == "COMPONENT1"]

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,TEST_0000,COMPONENT1,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
11,TEST_0011,COMPONENT1,2033,2016,200,0,0,2,4,27,0.0,0,174,0,15,0,0,127.1,1188
12,TEST_0012,COMPONENT1,2369,2013,200,0,0,1,2,14,0.0,0,41,0,40,0,0,118.0,994
18,TEST_0018,COMPONENT1,2415,2021,565,0,0,1,2,34,0.0,0,0,0,5,0,0,87.8,1078
20,TEST_0020,COMPONENT1,2032,2013,200,0,0,5,15,62,0.0,0,147,0,11,0,0,111.2,1161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6030,TEST_6030,COMPONENT1,8537,2022,364,0,0,0,0,10,0.0,0,8,0,5,0,0,100.1,1194
6031,TEST_6031,COMPONENT1,1775,2015,200,0,0,2,1,15,0.0,1,40,0,19,0,0,101.3,994
6033,TEST_6033,COMPONENT1,1177,2018,375,0,0,1,2,16,0.0,0,35,0,5,0,0,95.4,992
6035,TEST_6035,COMPONENT1,2781,2013,200,0,0,2,4,29,0.0,0,225,0,10,0,0,79.0,1357
